# Group Project: Movie Recommendations (2487-T2 Machine Learning) [Group 2]
- Nova School of Business and Economics, Portugal
- Instructor: Qiwei Han, Ph.D.
- Program: Masters Program in Business Analytics
- Group Members: 
    - **Luca Silvano Carocci (53942)**
    - **Fridtjov Höyerholt Stokkeland (52922)**
    - **Diego García Rieckhof (53046)**
    - **Matilde Pesce (53258)**
    - **Florian Fritz Preiss (54385)**<br>

---
# Phase 2: Data Understanding [02 Data Wrangling]

This script processes the data collected in notebook 01_data-collection into two tables for conducting an Exploratory Data Analysis (EDA) and Feature Engineering.
1. **Ratings Table**: achieved by pre-processing 'ratings.csv' from GroupLens (available at https://grouplens.org/datasets/movielens/)
2. **Movies Table**: achieved by pre-processing 'movies.csv' and 'tags.csv' from GroupLens (available at https://grouplens.org/datasets/movielens/) as well as 'tmdb.csv' collected from TMDB (available at https://www.themoviedb.org/)

In [1]:
import os
import re
import ast
import string
import nltk
import pandas as pd
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from datetime import datetime
from collections import defaultdict

## 1. Movies Table
### 1.1 Import and Pre-Process Basic Movies Table

In [2]:
# Import movies table
movies_df = pd.read_csv('../00_Data/00_raw/movies.csv', dtype={'movieId': object})

# Remove potential duplicates
movies_df = movies_df.drop_duplicates()

# Generate year column
movies_df['title'] = movies_df['title'].str.strip()
movies_df['year'] = movies_df['title'].apply(lambda x: np.nan if re.search('\W\d\d\d\d\W', x[-6:]) == None else x[-5:-1])

# Generate list from genres column
movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (1995),[Comedy],1995


### 1.2 Add TMDB IDs to the Movies Table
The TMDB IDs will help enriching the movies_df with external data obtained via the TMDB API (https://developers.themoviedb.org)

In [3]:
# Import Links table
links_df = pd.read_csv('../00_Data/00_raw/links.csv', dtype={'movieId': object, 'imdbId': object, 'tmdbId': object})
links_df = links_df.drop_duplicates()
links_df = links_df.drop('imdbId', axis=1)
links_df['tmdbId'] = links_df['tmdbId'].str.strip()

# Merge Movies Table and Links Table to add tmdbIds to Movies Table
movies_df = movies_df.merge(links_df, on='movieId', how='left')
movies_df.head()

,movieId,title,genres,year,tmdbId
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,862
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,8844
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,15602
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,31357
4,5,Father of the Bride Part II (1995),[Comedy],1995,11862


### 1.3 Get All Tags Assigned to Each Movie from the Tags Table

In [4]:
# import tags table
tags_df = pd.read_csv('../00_Data/00_raw/tags.csv',
                      dtype={'userId': object, 'movieId': object})
# convert all tags into one word
tags_df['tag'] = tags_df['tag'].str.replace(' ', '')
tags_df.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,darkcomedy,1573943598
3,4,1732,greatdialogue,1573943604
4,4,7569,sobadit'sgood,1573943455


In [5]:
# Get all tags for each movie into a list
tags_by_movie = tags_df.groupby('movieId').agg({'tag': lambda x: np.unique([str(tag).lower() for tag in x])})
tags_by_movie.reset_index(inplace=True)

# merge list of tags to movies table
movies_df = movies_df.merge(tags_by_movie, on='movieId', how='left')
movies_df.head()

,movieId,title,genres,year,tmdbId,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,862,"[2009reissueinstereoscopic3-d, 3d, 55movieseve..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,8844,"[adaptationofbook, adaptedfrom:book, adventure..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,15602,"[annmargaret, bestfriend, burgessmeredith, clv..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,31357,"[basedonnovelorbook, characters, chickflick, c..."
4,5,Father of the Bride Part II (1995),[Comedy],1995,11862,"[aging, baby, clv, comedy, confidence, contrac..."


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62423 entries, 0 to 62422
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  object
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
 3   year     62012 non-null  object
 4   tmdbId   62316 non-null  object
 5   tag      45251 non-null  object
dtypes: object(6)
memory usage: 3.3+ MB


### 1.4 Get all Data from TMDB Table and add to Movies Table

In [7]:
# Import detailed movie datasets
tmdb_df = pd.read_csv('../00_Data/00_raw/tmdb.csv', lineterminator='\n', dtype={'tmdbId':object}).drop('Unnamed: 0', axis=1)
tmdb_df.rename(columns={'title':'title_tmdb'}, inplace=True)
tmdb_df = tmdb_df.replace('[]', np.nan) # Some cells are filled with an empty list
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], format='%Y-%m-%d')

# Merge TMDB data with all features from the main movies dataframe on the tmdbId
movies_df = movies_df.merge(tmdb_df, on='tmdbId', how='left')
movies_df.head()

,movieId,title,genres,year,tmdbId,tag,title_tmdb,release_date,collection_name,original_language,description,runtime,actors,director,production_countries,spoken_languages
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,862,"[2009reissueinstereoscopic3-d, 3d, 55movieseve...",Toy Story,1995-10-30,Toy Story Collection,en,"Led by Woody, Andy's toys live happily in his ...",81.0,"['Tom Hanks', 'Annie Potts', 'Tim Allen']",['John Lasseter'],['US'],['en']
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,8844,"[adaptationofbook, adaptedfrom:book, adventure...",Jumanji,1995-12-15,Jumanji Collection,en,When siblings Judy and Peter discover an encha...,104.0,"['Bonnie Hunt', 'Kirsten Dunst', 'Robin Willia...",['Joe Johnston'],['US'],"['en', 'fr']"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,15602,"[annmargaret, bestfriend, burgessmeredith, clv...",Grumpier Old Men,1995-12-22,Grumpy Old Men Collection,en,A family wedding reignites the ancient feud be...,101.0,"['Daryl Hannah', 'Sophia Loren', 'Walter Matth...",['Howard Deutch'],['US'],['en']
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,31357,"[basedonnovelorbook, characters, chickflick, c...",Waiting to Exhale,1995-12-22,NaN,en,"Cheated on, mistreated and stepped on, the wom...",127.0,"['Giancarlo Esposito', 'Angela Bassett', 'Denn...",['Forest Whitaker'],['US'],['en']
4,5,Father of the Bride Part II (1995),[Comedy],1995,11862,"[aging, baby, clv, comedy, confidence, contrac...",Father of the Bride Part II,1995-12-08,Father of the Bride (Steve Martin) Collection,en,Just when George Banks has recovered from his ...,106.0,"['Kieran Culkin', 'Diane Keaton', 'BD Wong']",['Charles Shyer'],['US'],['en']


In [8]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62423 entries, 0 to 62422
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               62423 non-null  object        
 1   title                 62423 non-null  object        
 2   genres                62423 non-null  object        
 3   year                  62012 non-null  object        
 4   tmdbId                62316 non-null  object        
 5   tag                   45251 non-null  object        
 6   title_tmdb            54319 non-null  object        
 7   release_date          54297 non-null  datetime64[ns]
 8   collection_name       6096 non-null   object        
 9   original_language     54319 non-null  object        
 10  description           53921 non-null  object        
 11  runtime               54319 non-null  float64       
 12  actors                52358 non-null  object        
 13  director        

In [9]:
# replace year column with year of realease_date is not available
movies_df['year'] = movies_df['year'].apply(lambda x: movies_df['release_date'].year if x == np.nan else x)

# drop title_tmdb (title already available) and release_date (release year is sufficient)
movies_df.drop(['title_tmdb', 'release_date'], axis=1, inplace=True)

cols = list(movies_df.columns)

for col in ['collection_name', 'tag']:
	cols.remove(col)
    
movies_df.dropna(subset=cols, inplace=True)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48454 entries, 0 to 61872
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               48454 non-null  object 
 1   title                 48454 non-null  object 
 2   genres                48454 non-null  object 
 3   year                  48454 non-null  object 
 4   tmdbId                48454 non-null  object 
 5   tag                   39975 non-null  object 
 6   collection_name       5882 non-null   object 
 7   original_language     48454 non-null  object 
 8   description           48454 non-null  object 
 9   runtime               48454 non-null  float64
 10  actors                48454 non-null  object 
 11  director              48454 non-null  object 
 12  production_countries  48454 non-null  object 
 13  spoken_languages      48454 non-null  object 
dtypes: float64(1), object(13)
memory usage: 5.5+ MB


### 1.5 Pre-Processing of description column for EDA

In [10]:
# Set description column to lowercase
movies_df['description_cleaned'] = movies_df['description'].apply(lambda x: x.lower())
for pattern in ['\'s', '\"', '`', '´', '”', '’s', '’', 'n\'t', '‘|’', '–', '“', '—', '\'ll', '‘']:
    movies_df['description_cleaned'] = movies_df['description_cleaned'].str.replace(pattern, '', regex=False)
    movies_df['description_cleaned'] = movies_df['description_cleaned'].str.replace(r'\s*\.{3}\s*', ' ', regex=True)

# Remove stopwords from description column
stop_list = set(stopwords.words('english') + list(punctuation))
for idx, rows in movies_df.iterrows():
    word_tokens = nltk.tokenize.word_tokenize(movies_df.loc[idx,'description_cleaned'])
    movies_df.loc[idx,'description_cleaned_wsw'] = ' '.join([w for w in word_tokens if not w.lower() in stop_list])

In [11]:
# Basic calculations for description column
movies_df['description_nwords'] = movies_df['description_cleaned'].apply(lambda x: len(re.split(' ',x)))
movies_df['description_nwords_wsw'] = movies_df['description_cleaned_wsw'].apply(lambda x: len(re.split(' ',x)))
movies_df['description_nwords_unique'] = movies_df['description_cleaned_wsw'].apply(lambda x: len(set(re.split(' ',x))))
movies_df['description_nstopwords'] = movies_df['description_nwords'] - movies_df['description_nwords_wsw']
movies_df['description_diff_nwords'] = movies_df['description_nwords'] - movies_df['description_nstopwords']
movies_df['description_meanword'] = movies_df['description_cleaned'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
movies_df['description_meanword_wsw'] = movies_df['description_cleaned_wsw'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
movies_df['description_nchars'] = movies_df['description_cleaned'].apply(lambda x: len(str(x)))
movies_df['description_nchars_wsw'] = movies_df['description_cleaned_wsw'].apply(lambda x: len(str(x)))
movies_df['description_diff_nchars'] = movies_df['description_nchars'] - movies_df['description_nchars_wsw']

In [12]:
# Transform description column: lemmatization, root words
lemmatizer  = nltk.stem.WordNetLemmatizer()
root = []
for idx, rows in movies_df.iterrows():
    root_words = []
    words = movies_df.loc[idx,'description_cleaned_wsw'].split()
    for wrd in words:
        root_words.append(lemmatizer.lemmatize(wrd))
    root.append(' '.join(root_words))
movies_df['description_root_wrds'] = root

# Part of speech tagging (POS) for description column
container = []
for idx, rows in movies_df.iterrows():
    pos_cont = []
    text = movies_df.loc[idx, 'description_cleaned']
    tokens = nltk.tokenize.word_tokenize(text)
    for token in tokens:
        pos_cont.append(dict([nltk.pos_tag([token])[0][::-1]]))
    dd = defaultdict(list)
    for d in pos_cont:
        for key, value in d.items():
            dd[key].append(value)
    container.append(dd)
movies_df.loc[:, 'description_postag'] = container

# Compute number of tags for description column
dict_list = []
for idx, rows in movies_df.iterrows():
    d = movies_df.loc[idx,'description_postag']
    dict_base = {}
    for key, value in d.items():
        dict_base[key+'_n'] = len(d[key])
    dict_list.append(dict_base)

tempDf = pd.DataFrame.from_records(dict_list)

# count of adjectives (JJ) in the movie descriptions
movies_df['description_jj_n'] = tempDf[[i for i in tempDf.columns if 'JJ' in i]].sum(axis=1)
# count of nouns (NN) in the movie descriptions
movies_df['description_nn_n'] = tempDf[[i for i in tempDf.columns if 'NN' in i]].sum(axis=1)
# count of pronouns (PRP) in the movie descriptions
movies_df['description_prp_n'] = tempDf[[i for i in tempDf.columns if 'PRP' in i]].sum(axis=1)
# count of adverbs (RB) in the movie descriptions
movies_df['description_rb_n'] = tempDf[[i for i in tempDf.columns if 'RB' in i]].sum(axis=1)
# count of verbs (VB) in the movie descriptions
movies_df['description_vb_n'] = tempDf[[i for i in tempDf.columns if 'VB' in i]].sum(axis=1)

movies_df.drop(['description_postag'],inplace=True,axis=1)
movies_df.head()

,movieId,title,genres,year,tmdbId,tag,collection_name,original_language,description,runtime,...,description_meanword_wsw,description_nchars,description_nchars_wsw,description_diff_nchars,description_root_wrds,description_jj_n,description_nn_n,description_prp_n,description_rb_n,description_vb_n
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,862,"[2009reissueinstereoscopic3-d, 3d, 55movieseve...",Toy Story Collection,en,"Led by Woody, Andy's toys live happily in his ...",81.0,...,5.575758,297,216,81,led woody andy toy live happily room andy birt...,2.0,25.0,4.0,4.0,2.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,8844,"[adaptationofbook, adaptedfrom:book, adventure...",Jumanji Collection,en,When siblings Judy and Peter discover an encha...,104.0,...,5.675000,391,266,125,sibling judy peter discover enchanted board ga...,3.0,27.0,3.0,3.0,7.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,15602,"[annmargaret, bestfriend, burgessmeredith, clv...",Grumpy Old Men Collection,en,A family wedding reignites the ancient feud be...,101.0,...,6.060606,322,232,90,family wedding reignites ancient feud next-doo...,5.0,24.0,3.0,4.0,2.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,31357,"[basedonnovelorbook, characters, chickflick, c...",NaN,en,"Cheated on, mistreated and stepped on, the wom...",127.0,...,6.461538,268,193,75,cheated mistreated stepped woman holding breat...,3.0,15.0,2.0,1.0,8.0
4,5,Father of the Bride Part II (1995),[Comedy],1995,11862,"[aging, baby, clv, comedy, confidence, contrac...",Father of the Bride (Steve Martin) Collection,en,Just when George Banks has recovered from his ...,106.0,...,6.714286,291,161,130,george bank recovered daughter wedding receive...,1.0,18.0,6.0,3.0,7.0


In [13]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48454 entries, 0 to 61872
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movieId                    48454 non-null  object 
 1   title                      48454 non-null  object 
 2   genres                     48454 non-null  object 
 3   year                       48454 non-null  object 
 4   tmdbId                     48454 non-null  object 
 5   tag                        39975 non-null  object 
 6   collection_name            5882 non-null   object 
 7   original_language          48454 non-null  object 
 8   description                48454 non-null  object 
 9   runtime                    48454 non-null  float64
 10  actors                     48454 non-null  object 
 11  director                   48454 non-null  object 
 12  production_countries       48454 non-null  object 
 13  spoken_languages           48454 non-null  obj

In [14]:
movies_df.to_csv('../00_Data/01_processed/prepr_movies.csv')

## 2. Ratings Table

In [15]:
ratings_df = pd.read_csv('../00_Data/00_raw/ratings.csv',
                         dtype={'userId': object, 'movieId': object})
ratings_df = ratings_df.drop_duplicates()
# drop all ratings of movies where data is inconsistent
ratings_df = ratings_df[ratings_df['movieId'].isin(movies_df['movieId'])]
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [16]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24734765 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     object 
 1   movieId    object 
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 943.6+ MB


In [17]:
ratings_df.to_csv('../00_Data/01_processed/prepr_ratings.csv')

## 3. Result
The resulting two tables have the following structre:
* **Pre-Processed Ratings Table (prepr_ratings.csv)**: A table containing ~25M ratings of ~50k movies by ~160k users
	* 'userId': The unique identifier of the user who submitted the rating.
    * 'movieId': The unique identifier of each movie.
    * 'rating': The rating the user assigned to the movie between 0.5 (lowest) and 5.0 (highest) in 0.5 intervals.
    * 'timestamp': The date and time at which the rating has been submitted.
* **Pre-Processed Movies Table (prepr_movies.csv)**: A table containing detailed information on all of the ~50k rated movies
	* 'movieId': The unique identifier of each movie.
    * 'title': The title of the movie.
    * 'genres': A list of genres the movie has been assigned to.
    * 'year': The year of publication of the movie.
    * 'tmdbId': The unique identifier of each movie in the TMDB database (https://www.themoviedb.org/).
    * 'tag': A list of tags that have been assigned to each movie (a tag is assigned to a movie if at least one user has assigned the tag to the movie)
    * 'collection_name': The name of the collection the movie is part of (NaN-values are not part of any collection)
    * 'original_language': The original language of the movie.
    * 'description': A short description of the movie plot.
    * 'runtime': The duration of the movie in minutes.
    * 'actors': A list containing the top 3 most popular actors starring in the movie according to the popularity index by TMDB (https://www.themoviedb.org/).
    * 'director': The director(s) of the movie.
    * 'production_countries': The countries in which the movie has been produced.
    * 'spoken_languages': A list of the languages spoken in the movie.
    * 'description_cleaned': A cleaned version of the description text.
    * 'description_cleaned_wsw': The cleaned description text with all stop-words removed.
    * 'description_nwords': The wordcount of the movie plot description text.
    * 'description_nwords_wsw': The wordcount of the movie plot description text without excluding stopwords.
    * 'description_nwords_unique': The count of unique words used in the movie plot description.
    * 'description_nstopwords': The count of stopwords used in the movie plot description.
    * 'description_diff_nwords': The difference between the number of words in 'description_nwords' and the number of stopwords.
    * 'description_meanword': The average word length in the 'description_cleaned' column.
    * 'description_meanword_wsw': The average word length in the 'description_cleaned_wsw' column which has stopwords removed
    * 'description_nchars':  The total number of characters in the 'description_cleaned' column.
    * 'description_nchars_wsw': The total number of characters in the 'description_cleaned_wsw' column which has stopwords removed.
    * 'description_diff_nchars': The difference between the number of characters in 'description_nchars' and the number of characters in 'description_nchars_wsw'.
    * 'description_root_wrds': The root words (also known as base words, stems, or lemmas) for the words in the movie descriptions.
    * 'description_jj_n': The total count of adjectives (JJ) in the movie descriptions.
    * 'description_nn_n': The total count of nouns (NN) in the movie descriptions. 
    * 'description_prp_n': The total count of pronouns (PRP) in the movie descriptions.
    * 'description_rb_n': The total count of adverbs (RB) in the movie descriptions. 
    * 'description_vb_n': The total count of verbs (VB) in the movie descriptions. 